In [111]:

import numpy as np
import pandas as pd
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui,QtCore
from PyQt5.QtWidgets import QApplication,QGraphicsRectItem
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib.backends.backend_pdf import PdfPages
from math import ceil

global line_pos
line_pos = []

# Define the HistoWidget using pyqtgraph
class HistoWidget(pg.PlotWidget):

    rightClickSignal = QtCore.pyqtSignal(float, float)
    middleClickSignal = QtCore.pyqtSignal(float, float)
    leftClickSignal = QtCore.pyqtSignal(float, float)

    def __init__(self, data):
        super().__init__()
        self.data = data
        self.linemade = False

        self.rightClickSignal.connect(self.handleRightClick)
        self.middleClickSignal.connect(self.handleMiddleClick)
        self.leftClickSignal.connect(self.handleLeftClick)

        self.initHist()

    def initHist(self):
        cdata = [x for x in self.data if x < np.percentile(self.data,99)] 
        hy, hx = np.histogram(cdata, bins=50)
        self.plot(x=hx, y=hy, 
                  stepMode=True,
                  width=5)
        
        self.plotItem.vb.setLimits( xMin=0,
                                    xMax=max(hx),
                                    yMin=0,
                                    yMax=max(hy))
        
        self.setXRange(0, np.percentile(data,99))

    def mousePressEvent(self, event):
        if event.button() == QtCore.Qt.RightButton:
            # Get mouse position
            pos = event.pos()
            # Convert mouse position to plot coordinates
            plot_pos = self.plotItem.vb.mapSceneToView(pos)
            # Emit the signal with x and y coordinates
            self.rightClickSignal.emit(plot_pos.x(), plot_pos.y())

        if event.button() == QtCore.Qt.MiddleButton:
            # Get mouse position
            pos = event.pos()
            # Convert mouse position to plot coordinates
            plot_pos = self.plotItem.vb.mapSceneToView(pos)
            # Emit the signal with x and y coordinates
            self.middleClickSignal.emit(plot_pos.x(), plot_pos.y())

        if event.button() == QtCore.Qt.LeftButton:
            # Get mouse position
            pos = event.pos()
            # Convert mouse position to plot coordinates
            plot_pos = self.plotItem.vb.mapSceneToView(pos)
            # Emit the signal with x and y coordinates
            self.leftClickSignal.emit(plot_pos.x(), plot_pos.y())


    def handleRightClick(self, x, y):
        # Handle the right-click event
        #print(f"Right-clicked at x: {x}, y: {y}")

        if hasattr(self, 'line'):
            # Update the position of the existing line
            self.line.setValue(x)
        else:
            # Create a new line if it doesn't exist
            self.line = pg.InfiniteLine(pos=x,
                                        angle=90,
                                        pen=pg.mkPen(color='b', width=5))
            self.addItem(self.line)
        

    def handleMiddleClick(self, x, y):
        # Handle the right-click event
        #print(f"Middle-clicked at x: {x}, y: {y}")

        if hasattr(self, 'line2'):
            # Update the position of the existing line
            self.line2.setValue(x)
        else:
            # Create a new line if it doesn't exist
            self.line2 = pg.InfiniteLine(pos=x,
                                        angle=90,
                                        pen=pg.mkPen(color='purple', width=5))
            self.addItem(self.line2)

    def handleLeftClick(self, x, y):
        # Handle the right-click event
        #print(f"Left-clicked at x: {x}, y: {y}")

        if hasattr(self, 'line3'):
            # Update the position of the existing line
            self.line3.setValue(x)
        else:
            # Create a new line if it doesn't exist
            self.line3 = pg.InfiniteLine(pos=x,
                                        angle=90,
                                        pen=pg.mkPen(color='red', width=5))
            self.addItem(self.line3)

    def keyPressEvent(self, event):
        # Handle key press events
        if event.key() == QtCore.Qt.Key_Enter or event.key() == QtCore.Qt.Key_Return:
            global line_pos
            line_pos.append([self.line.pos(),self.line2.pos(),self.line3.pos()])
            self.close()




# Function to display the histogram using pyqtgraph
def display_histogram(data):
    app = QApplication([])  # Create a Qt application instance
    win = HistoWidget(data)
    win.show()
    app.exec_()  # Start the Qt event loop

# Load data and display histogram
path = r"R:\data_analysis\Tyler\hgsc\data\hgsc-full-r2\oc3-cbpp\palbociclib.csv"
df = pd.read_csv(path)
df['integrated-DNA'] = df['area'] * df['DAPI-mean']

perc_vals = []

for ind,condition in enumerate(np.unique(df['condition'])):
    new_df = df[df['condition'] == condition]
    data = new_df['integrated-DNA']

    display_histogram(data)
    
    st,mid,end = np.sort([x[0] for x in line_pos[ind]])
    
    perc_2n = len(new_df[new_df['integrated-DNA'] < mid])/len(new_df)
    print(ind,perc_2n)

    perc_vals.append(perc_2n)



0 0.6324269889224572
1 0.50390717499408
2 0.5575168107588857
3 0.5165728580362726
4 0.5965628722137145
5 0.647433346597692
6 0.6284018651971174
7 0.6102748165211336


In [113]:
c_to_use = [0,2,3,6]

fig,axs = plt.subplots(len(c_to_use),1,sharex=True,figsize=(10,10))
plt.suptitle('_'.join(path.split('/')[-3:]))
ax_num=0

for i,x in enumerate(list(np.unique(df['condition']))):

    if i not in c_to_use or len(c_to_use)==0: continue
    axi=axs[ax_num]

    newd = (df[df['condition'] == x])['integrated-DNA']
    cnewd = newd[newd < np.percentile(newd,99)]
    normnewd = cnewd/(line_pos[i][0][0]/2)

    midpoint = 2*(line_pos[i][1][0]/line_pos[i][0][0])
    endpoint = 2*(line_pos[i][2][0]/line_pos[i][0][0])

    perc_2n = len([y for y in normnewd if y <= midpoint])/len(normnewd)*100
    perc_4n = len([y for y in normnewd if y >= midpoint and y <endpoint])/len(normnewd)*100
    perc_anu = len([y for y in normnewd if y >endpoint])/len(normnewd)*100

    if len(cnewd) > 10000: bcount=50
    else: bcount=50

    sns.histplot([y for y in normnewd if y < 8],bins=bcount,ax=axi,zorder=3,color='lightblue')
    axi.set_yticks([])
    plt.xlim(0,8)


    axi.axvspan(1, 3, color=(.05,.05,.05,.01), alpha=0.1,zorder=1)
    axi.axvspan(3, 5, color=(.05,.05,.05,.01), alpha=0.3,zorder=1)

    axi.text(1.05, axi.get_ylim()[1] * 0.95, f'2n: {ceil(perc_2n)}%', color='black', fontsize=8, ha='left', va='top')
    axi.text(3.05, axi.get_ylim()[1] * 0.95, f'4n: {ceil(perc_4n)}%', color='black', fontsize=8, ha='left', va='top')
    axi.text(5.05, axi.get_ylim()[1] * 0.95, f'>4n: {ceil(perc_anu)}%', color='black', fontsize=8, ha='left', va='top')

    ax_num = ax_num+1

cl = (path.split('\\')[-2]).split('-')[0]
dr = (''.join(path.split('\\')[-1][:-4]))
npath = os.path.join(r'R:\data_analysis\Tyler\hgsc\data\figures',cl)
sn = os.path.join(npath,f'int-dna-{dr}.pdf')

with PdfPages(sn) as pdf:
    pdf.savefig(fig)
    plt.close()